# 2. Preprocessing:

We first repeat the import steps to get all the df and update the schema

In [69]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import json

In [70]:
# Import df
END_POINT = "data.cityofnewyork.us"
NEW_APP_TOKEN = 'u9SAL7T0QJyF6aY8OsLyS7gBX'
USERNAME = 'b.gutu@columbia.edu'
PASSWORD = 'M#-wMw6;]&]kU@r'



client = Socrata(END_POINT,
                 NEW_APP_TOKEN,
                 username=USERNAME,
                 password=PASSWORD)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("7x5e-2fxh",limit=29000) #feel free to unlimit this if you'd like. 2000 is sufficient for the hackathon (if not doing ML)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)


Update the schema

In [79]:
# Adjusting df types
# print(list(df.dtypes))

types = {'numeric':0,'text':0,'floating_timestamp':0}
from schema import schema_dict

for column, dtype in schema_dict.items():
    if column not in df.columns:
        print('col not present',column)
        continue
    if dtype == 'number':
        types['numeric'] +=1
        df[column] = df[column].astype(float)  # or .astype(int) depending on the specifics
    elif dtype == 'text':
        types['text'] +=1

        df[column] = df[column].astype(str)
    elif dtype == 'floating_timestamp':
        types['floating_timestamp'] +=1

        df[column] = pd.to_datetime(df[column])

types


col not present bin
col not present bbl
col not present dof_benchmarking_submission_id


{'numeric': 15, 'text': 230, 'floating_timestamp': 3}

In [80]:
# Before filling
# Find the percentage of missing values for each column
missing_percent = (df.isnull().sum() / len(df)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)



latitude            3.789655
longitude           3.789655
community_board     3.789655
council_district    3.789655
census_tract        3.789655
borough             3.768966
dtype: float64


In [81]:
# drop all of the na
df.dropna(inplace=True)
missing_percent = (df.isnull().sum() / len(df)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)


Series([], dtype: float64)


In [73]:
df.replace("Not Available", pd.NA, inplace=True)
df.replace("Not Applicable", pd.NA, regex=True, inplace=True)

# Find the percentage of missing values for each column
missing_percent = (df.isnull().sum() / len(df)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)


propane_use_kbtu                   100.000000
barracks_gross_floor_area          100.000000
data_center_national_median         99.996552
data_center_pue                     99.996552
convention_center_gross_floor       99.993103
district_hot_water_use_kbtu         99.986207
municipally_supplied_potable_3      99.975862
outdoor_water_use_all_water         99.975862
wholesale_club_supercenter          99.962069
third_party_certification_2         99.962069
third_party_certification_1         99.962069
third_party_certification           99.962069
wholesale_club_supercenter_1        99.962069
energy_power_station_gross          99.955172
ambulatory_surgical_center          99.955172
fuel_oil_1_use_kbtu                 99.955172
wastewater_treatment_plant          99.951724
estimated_data_flag_fuel            99.948276
fuel_oil_5_6_use_kbtu               99.941379
automobile_dealership_gross         99.917241
adult_education_gross_floor         99.913793
mailing_center_post_office        

In [134]:
# Save the DataFrame to a CSV file
df.to_csv('./data/data_full.csv', index=False)

# Get the schema of the DataFrame
schema = {
    column: str(dtype) 
    for column, dtype in zip(df.columns, df.dtypes)
}

# Write the schema to a JSON file
with open('./data/schema.json', 'w') as schema_file:
    json.dump(schema, schema_file, indent=4)

# Focusing on multifamily

In [75]:
df_saved = pd.read_csv('./data/data_full.csv')

C:\Users\andre\AppData\Local\Temp\ipykernel_31780\2101965794.py:1: DtypeWarning: Columns (9,30,32,54,55,57,58,60,61,67,68,97,98,99,100,121,192,227,228,229) have mixed types. Specify dtype option on import or set low_memory=False.
  df_saved = pd.read_csv('./data/data_full.csv')


In [125]:
# Drop all columns where multifam is na. We will use 'multifamily_housing_number' as a surrogate

surrogates = ['multifamily_housing_total']
# Assuming df is your DataFrame and 'column_name' is the name of the column
for surrogate in surrogates:
    df_saved.dropna(subset=[surrogate],inplace=True)


# See the new nas
# Find the percentage of missing values for each column
missing_percent = (df_saved.isnull().sum() / len(df_saved)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)


distribution_center_gross          100.000000
barracks_gross_floor_area          100.000000
convention_center_gross_floor      100.000000
refrigerated_warehouse_gross       100.000000
courthouse_gross_floor_area        100.000000
hospital_general_medical_2         100.000000
wastewater_treatment_plant         100.000000
district_hot_water_use_kbtu        100.000000
propane_use_kbtu                   100.000000
data_center_pue                    100.000000
data_center_national_median        100.000000
hospital_general_medical_1          99.994708
wholesale_club_supercenter          99.994708
data_center_cooling_equipment       99.994708
data_center_ups_system              99.994708
automobile_dealership_gross         99.994708
wholesale_club_supercenter_1        99.994708
manufacturing_industrial            99.994708
hospital_general_medical_6          99.994708
hospital_general_medical            99.994708
hospital_general_medical_3          99.994708
hospital_general_medical_4        

In [126]:
# DROP

threshold_percentage = 2
missing_percent = (df_saved.isnull().sum() / len(df_saved)) * 100
# Get columns to drop based on threshold
columns_to_drop = missing_percent[missing_percent > threshold_percentage].index

print('dropping ', len(list(columns_to_drop)))

len(df_saved.columns)

# Drop these columns from the DataFrame
df_cleaned = df_saved.drop(columns=columns_to_drop)

len(df_cleaned.columns)

dropping  187


62

In [127]:
# See the new nas
# Find the percentage of missing values for each column
missing_percent = (df_cleaned.isnull().sum() / len(df_cleaned)) * 100
# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)


metered_areas_energy             1.693390
indirect_ghg_emissions_metric    0.952532
indirect_ghg_emissions           0.952532
net_emissions_metric_tons        0.867863
nyc_building_identification      0.127004
nyc_borough_block_and_lot        0.116421
egrid_output_emissions_rate      0.105837
national_median_total_ghg        0.047627
last_modified_date_property_1    0.037043
last_modified_date_property      0.031751
national_median_source_eui       0.010584
national_median_site_eui         0.010584
national_median_reference        0.010584
multifamily_housing_number_7     0.010584
multifamily_housing_number_1     0.005292
multifamily_housing_total_1      0.005292
multifamily_housing_number_5     0.005292
multifamily_housing_number_6     0.005292
multifamily_housing_percent      0.005292
multifamily_housing_percent_1    0.005292
dtype: float64


In [128]:
df_cleaned.dropna(inplace=True)

In [131]:
df_cleaned

,property_id,property_name,year_ending,nyc_borough_block_and_lot,nyc_building_identification,address_1,city,postal_code,primary_property_type_self,primary_property_type,...,property_gfa_calculated,property_gfa_calculated_1,last_modified_date_property,last_modified_date_property_1,number_of_active_energy_meters,number_of_active_energy_meters_1,number_of_active_energy_meters_2,number_of_active_it_meters,generation_date,submission_date
0,21205224.0,Astoria Blvd Property,2021-12-31,4006520042,4538607,36-16 Astoria Blvd.,Long Island City,11103,Multifamily Housing,Multifamily Housing,...,34146.0,25000.0,07/11/2022 11:06 AM EDT,05/19/2022 09:25 AM EDT,8.0,8.0,0.0,0.0,2022-09-26,2022-09-26
1,2665352.0,Stellar - 70 West 93rd Street,2021-12-31,1-01206-0001,1031627,70 West 93rd Street,New York,10025,Multifamily Housing,Multifamily Housing,...,261980.0,261980.0,09/12/2022 04:34 PM EDT,04/10/2022 03:59 PM EDT,6.0,6.0,0.0,0.0,2022-09-26,2022-09-26
2,2665400.0,Stellar - 28-50 West 97th Street,2021-12-31,1-01832-0043,1055265,28-50 West 97th Street,New York,10025,Multifamily Housing,Multifamily Housing,...,363092.0,333342.0,04/09/2022 06:13 PM EDT,08/05/2014 10:15 PM EDT,3.0,3.0,0.0,0.0,2022-09-26,2022-09-26
3,2665405.0,Stellar - 310 Greenwich St,2021-12-31,1-00142-0025,1083157;1083235;1083237;1083240;1083241;108324...,310 Greenwich Street / 40 Harrison Street,New York,10013,Multifamily Housing,Multifamily Housing,...,1345900.0,1112900.0,09/12/2022 04:33 PM EDT,08/25/2019 09:27 AM EDT,4.0,4.0,0.0,0.0,2022-09-26,2022-09-26
4,2665415.0,Stellar - 87 Hamilton Place,2021-12-31,1-02072-0039,1061772,87 Hamilton Place,New York,10031,Multifamily Housing,Multifamily Housing,...,72662.0,72662.0,05/22/2022 10:41 PM EDT,04/04/2020 11:47 PM EDT,11.0,11.0,0.0,0.0,2022-09-26,2022-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28994,6575086.0,35 WINTHROP REALTY CORP,2021-12-31,3050450091,3115687;3115686,35 WINTHROP STREET,BROOKLYN,11225,Multifamily Housing,Multifamily Housing,...,38199.0,38199.0,03/02/2022 07:42 PM EST,03/19/2020 06:21 PM EDT,4.0,4.0,0.0,0.0,2022-03-29,2022-03-29
28995,6575119.0,782 EAST 32ND STREET LLC,2021-12-31,3075590001,3205933,782 EAST 32ND STREET,BROOKLYN,11210,Multifamily Housing,Multifamily Housing,...,38736.0,38736.0,03/02/2022 07:42 PM EST,04/05/2020 11:17 AM EDT,4.0,4.0,0.0,0.0,2022-03-29,2022-03-29
28996,6575799.0,TAG REALTY INC,2021-12-31,3012800066,3033472,969 CARROLL STREET,BROOKLYN,11225,Multifamily Housing,Multifamily Housing,...,35824.0,35824.0,03/02/2022 08:10 PM EST,03/19/2020 06:28 PM EDT,4.0,4.0,0.0,0.0,2022-03-29,2022-03-29
28997,6576684.0,LINCOLN64 FLATS LLC,2021-12-31,3050280006,3114678,64 LINCOLN ROAD,BROOKLYN,11225,Multifamily Housing,Multifamily Housing,...,49930.0,49930.0,03/02/2022 07:42 PM EST,07/24/2020 12:57 AM EDT,7.0,7.0,0.0,0.0,2022-03-29,2022-03-29


# Save the data
We have dealt with initial features and data by looking at the N/As and equivalent values in the data.
We have filtered to only multi-family homes as well. There is bias from doing this which is unavoidable as the only way to truly know the right action is to look at each feature.

From here, we will save the data and perform EDA to consider our next steps.


In [135]:
# Save the DataFrame to a CSV file
df_cleaned.to_csv('./data/data_cleaned.csv', index=False)

# Get the schema of the DataFrame
schema = {
    column: str(dtype) 
    for column, dtype in zip(df_cleaned.columns, df_cleaned.dtypes)
}

# Write the schema to a JSON file
with open('./data/schema_cleaned.json', 'w') as schema_file:
    json.dump(schema, schema_file, indent=4)

Demo to read the data       

In [136]:
import pandas as pd
import json

# Load the schema from the JSON file
with open('./data/schema_cleaned.json') as schema_file:
    schema = json.load(schema_file)

# Convert schema to the format required by pandas
# Note: In this case, we're assuming all the dtypes are compatible with pandas dtypes.
# If there are any discrepancies, you may need to manually adjust the dtypes.
pandas_schema = {key: value for key, value in schema.items()}

# Now read the CSV file using the schema
df_new = pd.read_csv('./data/data_cleaned.csv', dtype=pandas_schema)

# Now df has the schema as specified in the schema.json file
